In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('https://raw.githubusercontent.com/alco-845/ta-datamining/master/Banking%20Analyze.csv')
data

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,subscribed
0,26110,56,admin,married,unknown,no,1933,no,no,telephone,19,nov,44,2,-1,0,unknown,no
1,40576,31,unknown,married,secondary,no,3,no,no,cellular,20,jul,91,2,-1,0,unknown,no
2,15320,27,services,married,secondary,no,891,yes,no,cellular,18,jul,240,1,-1,0,unknown,no
3,43962,57,management,divorced,tertiary,no,3287,no,no,cellular,22,jun,867,1,84,3,success,yes
4,29842,31,technician,married,secondary,no,119,yes,no,cellular,4,feb,380,1,-1,0,unknown,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1063,18988,54,blue-collar,married,unknown,no,191,no,no,cellular,4,aug,753,2,-1,0,unknown,no
1064,18846,45,management,divorced,secondary,no,1950,no,no,cellular,4,aug,498,1,-1,0,unknown,no
1065,28886,46,blue-collar,married,primary,no,665,no,no,cellular,30,jan,89,4,235,2,failure,no
1066,32043,54,admin,married,secondary,no,3136,yes,no,telephone,14,apr,104,1,146,1,failure,no


In [11]:
### Modelling 
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

### Remove unnecessary warnings
import warnings
warnings.filterwarnings('ignore')

In [4]:
data_baru = data.drop(columns=["ID", "job", "contact", "day", "month", "poutcome", "subscribed"])
        
sebelum_dinormalisasi = ['age', 'balance', 'duration', 'campaign', 'pdays', 'previous']
setelah_dinormalisasi = ['Norm_age', 'Norm_balance', 'Norm_duration', 'Norm_campaign', 'Norm_pdays', 'Norm_previous']

normalisasi_fitur = data[sebelum_dinormalisasi]

scaler = MinMaxScaler()
scaler.fit(normalisasi_fitur)
fitur_ternormalisasi = scaler.transform(normalisasi_fitur)

fitur_ternormalisasi_df = pd.DataFrame(fitur_ternormalisasi, columns = setelah_dinormalisasi)

fitur_ternormalisasi

array([[0.53731343, 0.0820954 , 0.0236267 , 0.02702703, 0.        ,
        0.        ],
       [0.1641791 , 0.04119604, 0.05138807, 0.02702703, 0.        ,
        0.        ],
       [0.10447761, 0.06001399, 0.13939752, 0.        , 0.        ,
        0.        ],
       ...,
       [0.3880597 , 0.05522473, 0.05020673, 0.08108108, 0.27995255,
        0.125     ],
       [0.50746269, 0.10758863, 0.05906675, 0.        , 0.17437722,
        0.0625    ],
       [0.04477612, 0.04312446, 0.11163615, 0.        , 0.        ,
        0.        ]])

In [5]:
data.loc[(data['marital'] == "divorced") ,"marital"] = 2
data.loc[(data['marital'] == "married") ,"marital"] = 1
data.loc[(data['marital'] == "single"),"marital"] = 0

data.loc[(data['education'] == "tertiary"),"education"] = 2
data.loc[(data['education'] == "unknown") ,"education"] = 1
data.loc[(data['education'] == "secondary") ,"education"] = 1
data.loc[(data['education'] == "primary"),"education"] = 0

data['default'] = data['default'].replace({'yes': 1, 'no': 0})
data['housing'] = data['housing'].replace({'yes': 1, 'no': 0}) 
data['loan'] = data['loan'].replace({'yes': 1, 'no': 0})

data_baru = data.drop(columns=["ID", "age", "job", "balance", "contact", "day", "month", "duration", "campaign", "pdays", "previous", "subscribed"])

split_overdue = pd.get_dummies(data_baru["poutcome"], prefix="poutcome")
data_baru = data_baru.join(split_overdue)
data_baru = data_baru.drop(columns = "poutcome")

data_baru

,marital,education,default,housing,loan,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,1,1,0,0,0,0,0,0,1
1,1,1,0,0,0,0,0,0,1
2,1,1,0,1,0,0,0,0,1
3,2,2,0,0,0,0,0,1,0
4,1,1,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...
1063,1,1,0,0,0,0,0,0,1
1064,2,1,0,0,0,0,0,0,1
1065,1,0,0,0,0,1,0,0,0
1066,1,1,0,1,0,1,0,0,0


In [6]:
data_sudah_normal = data_baru
        
data_sudah_normal = data_sudah_normal.join(fitur_ternormalisasi_df)

data_sudah_normal

,marital,education,default,housing,loan,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown,Norm_age,Norm_balance,Norm_duration,Norm_campaign,Norm_pdays,Norm_previous
0,1,1,0,0,0,0,0,0,1,0.537313,0.082095,0.023627,0.027027,0.000000,0.0000
1,1,1,0,0,0,0,0,0,1,0.164179,0.041196,0.051388,0.027027,0.000000,0.0000
2,1,1,0,1,0,0,0,0,1,0.104478,0.060014,0.139398,0.000000,0.000000,0.0000
3,2,2,0,0,0,0,0,1,0,0.552239,0.110789,0.509746,0.000000,0.100830,0.1875
4,1,1,0,1,0,0,0,0,1,0.164179,0.043654,0.222091,0.000000,0.000000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1063,1,1,0,0,0,0,0,0,1,0.507463,0.045180,0.442410,0.027027,0.000000,0.0000
1064,2,1,0,0,0,0,0,0,1,0.373134,0.082456,0.291790,0.000000,0.000000,0.0000
1065,1,0,0,0,0,1,0,0,0,0.388060,0.055225,0.050207,0.081081,0.279953,0.1250
1066,1,1,0,1,0,1,0,0,0,0.507463,0.107589,0.059067,0.000000,0.174377,0.0625


In [8]:
X = data_sudah_normal.iloc[:,8:14]
Y = data.iloc[:,-1]

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.25, random_state=0)
X

,poutcome_unknown,Norm_age,Norm_balance,Norm_duration,Norm_campaign,Norm_pdays
0,1,0.537313,0.082095,0.023627,0.027027,0.000000
1,1,0.164179,0.041196,0.051388,0.027027,0.000000
2,1,0.104478,0.060014,0.139398,0.000000,0.000000
3,0,0.552239,0.110789,0.509746,0.000000,0.100830
4,1,0.164179,0.043654,0.222091,0.000000,0.000000
...,...,...,...,...,...,...
1063,1,0.507463,0.045180,0.442410,0.027027,0.000000
1064,1,0.373134,0.082456,0.291790,0.000000,0.000000
1065,0,0.388060,0.055225,0.050207,0.081081,0.279953
1066,0,0.507463,0.107589,0.059067,0.000000,0.174377


In [9]:
#===================== KNN =======================
knn = KNeighborsClassifier()
knn.fit(X_train, Y_train)
y_predknn = knn.predict(X_test)
knn_accuracy = round(100 * accuracy_score(Y_test, y_predknn), 2)

In [12]:
#===================== Bayes Gaussian =============
gaussian = GaussianNB()
gaussian.fit(X_train,Y_train)
y_pred_gaussian   =  gaussian.predict(X_test)
gauss_accuracy  = round(100*accuracy_score(Y_test, y_pred_gaussian),2)
gaussian_eval = classification_report(Y_test, y_pred_gaussian,output_dict = True)
gaussian_eval_df = pd.DataFrame(gaussian_eval).transpose()

In [13]:
#===================== Decission tree =============
decission3  = DecisionTreeClassifier(criterion="gini")
decission3.fit(X_train,Y_train)
y_pred_decission3 = decission3.predict(X_test)
decission3_accuracy = round(100*accuracy_score(Y_test, y_pred_decission3),2)
decission3_eval = classification_report(Y_test, y_pred_decission3,output_dict = True)
decission3_eval_df = pd.DataFrame(decission3_eval).transpose()

In [14]:
print("Dengan menggunakan metode KNN didapatkan akurasi sebesar:")
print(f"Akurasi  =  {knn_accuracy}%")

print("Dengan menggunakan metode Bayes Gaussian didapatkan hasil akurasi sebesar:")
print(f"Akurasi = {gauss_accuracy}%")

print("Dengan menggunakan metode Decission tree didapatkan hasil akurasi sebesar:")
print(f"Akurasi = {decission3_accuracy}%")

Dengan menggunakan metode KNN didapatkan akurasi sebesar:
Akurasi  =  87.64%
Dengan menggunakan metode Bayes Gaussian didapatkan hasil akurasi sebesar:
Akurasi = 86.52%
Dengan menggunakan metode Decission tree didapatkan hasil akurasi sebesar:
Akurasi = 82.02%


Export

In [15]:
from sklearn.utils.validation import joblib

joblib.dump(scaler, "scaler.joblib")

joblib.dump(knn, "knn.joblib")
joblib.dump(gaussian, "gaussian.joblib")
joblib.dump(decission3, "decission3.joblib")

['decission3.joblib']